# XRD Analysis

## Imports
* libraries 
* from python file

In [1]:
import pandas as pd
import fabio
import pyFAI

from XRD_methods_02 import *

import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State

## Files

build a class to keep everything together?

In [2]:
poni = 'ceria_callibration_geometry.poni'

In [3]:
rolled_file = "C:/Users/starlord/Desktop/Summer Research 2020/App/images/copper_as_rolled_003.tiff"
annealed_file = "C:/Users/starlord/Desktop/Summer Research 2020/App/images/copper_annealed_002.tiff"
hammered_file = "C:/Users/starlord/Desktop/Summer Research 2020/App/images/copper_hammered_002.tiff"

In [4]:
rolled = element(rolled_file)
annealed = element(annealed_file)
hammered = element(hammered_file)

I think I should add the parameters to be set during the creation of the element as it is useless without them and I will need to set them every time. 

## Parameters
* hkl
* lambda
* lattice side length
* spectrum dictionary with best guesses - write something to generate this

In [5]:
hkls = [
    [1, 1, 1],
    [2, 0, 0],
    [2, 2, 0],
    [3, 1, 1],
    [2, 2, 2],
    [4, 0, 0],
    [3, 3, 1],
    [4, 2, 0],
    [4, 2, 2],
    [3, 3, 3],
]

This is the wavelength of the x ray 

In [6]:
lamb1 = 0.3905 

This is the lattice parameter

In [7]:
a = 3.6150 

In [8]:
rolled.hkls = annealed.hkls = hammered.hkls = hkls

In [9]:
rolled.lamb = annealed.lamb = hammered.lamb = lamb1

In [10]:
rolled.a = annealed.a = hammered.a = a

There is nothing incredibly special about these functions so they might as well be run behind the scenes.

In [11]:
rolled.d = annealed.d = hammered.d = d_from_hkl(hkls, a)

In [12]:
rolled.tth = annealed.tth = hammered.tth = tth_from_d(rolled.d, rolled.lamb)

## Filter

A few of the pixels in the image are faulty and give us bad results. Filtering out these extreme values will give us more accurate results.
I believe this can also be done when creating the poni file. 

In [13]:
lim = 100000

rolled.filt(lim)
annealed.filt(lim)
hammered.filt(lim)

## Integrate

I feel comfortable leaving this outside of the class because it is a very important step. 

Maybe I can incorporate the filtering within this function. 

For some reason pyOpenCl still doesn't work. Fixing this should speed up calculations. 

In [14]:
rolled.res, rolled.res2 = process(rolled.data_filt, poni)

In [15]:
annealed.res, annealed.res2 = process(annealed.data_filt, poni)

In [16]:
hammered.res, hammered.res2 = process(annealed.data_filt, poni)

## Fit Peaks

Creat a model of the spectrum
this contains guesses where the peaks are and a histogram of the actual data

In [17]:
rolled.spec = create_spectrum(rolled.tth, rolled.res[0], rolled.res[1])

In [18]:
annealed.spec = create_spectrum(annealed.tth, annealed.res[0], annealed.res[1])

In [19]:
hammered.spec = create_spectrum(hammered.tth, hammered.res[0], hammered.res[1])

This updates using scipys find peaks function. it might be redundant 

In [20]:
rolled.peaks = update_spec_from_peaks(rolled.spec)

In [21]:
annealed.peaks = update_spec_from_peaks(annealed.spec)

In [22]:
hammered.peaks = update_spec_from_peaks(hammered.spec)

#### This is where the magic actually happens

Get the model and the params to use in the fit

In [23]:
rolled.model, rolled.params = generate_model(rolled.spec)

In [24]:
annealed.model, annealed.params = generate_model(annealed.spec)

In [25]:
hammered.model, hammered.params = generate_model(hammered.spec)

Perform the fit and extract the results.

The ycomps variable can be plotted over the histogram (vs the x of the histogram) it is a list with one component (array) for each peak 

the params are the centers and widths of each peak.

In [26]:
rolled.ycomps, rolled.best_values = get_output(
    rolled.model,
    rolled.params,
    rolled.res[0],
    rolled.res[1],
    rolled.spec,
)

In [27]:
annealed.ycomps, annealed.best_values = get_output(
    annealed.model,
    annealed.params,
    annealed.res[0],
    annealed.res[1],
    annealed.spec,
)

In [28]:
hammered.ycomps, hammered.best_values = get_output(
    hammered.model,
    hammered.params,
    hammered.res[0],
    hammered.res[1],
    hammered.spec,
)

## Generate Figs
* images
* histograms
* 2D histograms
* scatter polars


In [29]:
rolled.gen_figs()

In [30]:
annealed.gen_figs()

In [31]:
hammered.gen_figs()

In [32]:
#rolled.img_fig.show()
#rolled.hist_fig.show()

## Structure App

This is honestly just a lot of html and css

In [33]:
app = dash.Dash(external_stylesheets=[dbc.themes.LITERA])

### Navbar

In [34]:
navbar = dbc.Navbar([
    dbc.Row(
        [dbc.Col([dbc.NavbarBrand("XRD Analysis", className="navbar-brand")])])
],
                    color='primary',
                    className="navbar navbar-expand-lg navbar-dark bg-primary")

### Histograms

I am not sure if this step belongs here. Seems fitting for now

In [35]:
rolled.df = get_vals_df(rolled.spec, rolled.best_values)

In [36]:
annealed.df = get_vals_df(annealed.spec, annealed.best_values)

In [37]:
hammered.df = get_vals_df(hammered.spec, hammered.best_values)

In [38]:
rolled.df = rolled.df.round(decimals=3)
annealed.df = annealed.df.round(decimals=3)
hammered.df = hammered.df.round(decimals=3)

#### This will be the first page that loads. 
It has all of the important results as well as it is light on the rendering side.

In [39]:
histograms = html.Div([
    dbc.Row([
        dbc.Col([dcc.Graph(figure=rolled.hist_fig)], align='center'),
        dbc.Col([
            dbc.Table.from_dataframe(rolled.df,
                                     striped=True,
                                     bordered=True,
                                     className="table-secondary",
                                     size='sm'),
        ]),
    ]),
    html.Hr(className="my-4"),
    dbc.Row([
        dbc.Col([dcc.Graph(figure=annealed.hist_fig)], align='center'),
        dbc.Col([
            dbc.Table.from_dataframe(annealed.df,
                                     striped=True,
                                     bordered=True,
                                     className="table-secondary",
                                     size='sm'),
        ]),
    ]),
    html.Hr(className="my-4"),
    dbc.Row([
        dbc.Col([dcc.Graph(figure=hammered.hist_fig)], align='center'),
        dbc.Col([
            dbc.Table.from_dataframe(hammered.df,
                                     striped=True,
                                     bordered=True,
                                     className="table-secondary",
                                     size='sm'),
        ]),
    ]),
])

### Images

In [40]:
images = html.Div([
    dbc.Row([dbc.Col([dcc.Graph(figure=rolled.img_fig)], align='center')]),
    dbc.Row([dbc.Col([dcc.Graph(figure=annealed.img_fig)], align='center')]),
    dbc.Row([dbc.Col([dcc.Graph(figure=hammered.img_fig)], align='center')]),
])

### 2D Histograms

In [41]:
hists_2D = html.Div([
    dbc.Row([dbc.Col([dcc.Graph(figure=rolled.hist_fig_2D)], align='center')]),
    dbc.Row(
        [dbc.Col([dcc.Graph(figure=annealed.hist_fig_2D)], align='center')]),
    dbc.Row(
        [dbc.Col([dcc.Graph(figure=hammered.hist_fig_2D)], align='center')]),
])

### Polar Plots

In [42]:
polar_plots = html.Div([
    dbc.Row([
        dbc.Col([dcc.Graph(figure=rolled.polar_fig)],
                width={
                    "size": 8,
                    "offset": 2,
                },
                align='center')
    ]),
    dbc.Row([
        dbc.Col([dcc.Graph(figure=annealed.polar_fig)],
                width={
                    "size": 8,
                    "offset": 2
                },
                align='center')
    ]),
    dbc.Row([
        dbc.Col([dcc.Graph(figure=hammered.polar_fig)],
                width={
                    "size": 8,
                    "offset": 2
                },
                align='center')
    ]),
])

### Layout

In [43]:
tabs = dbc.Container([
    html.Hr(className="my-4"),
    dbc.Tabs([
        dbc.Tab(label="Histograms", tab_id="hist"),
        dbc.Tab(label="Polar Plots", tab_id="polar"),
        dbc.Tab(label="2D Histograms", tab_id="2D_hist"),
        dbc.Tab(label="Images", tab_id="images"),
    ],
             id="tabs",
             active_tab="hist",
             className="nav nav-tabs"),
    html.Hr(className="my-4"),
    html.Div(id="tab_content"),
])

In [44]:
app.layout = html.Div(children=[navbar,tabs])

## Callbacks 
* Tabs
* Regenerate Figs based on new filtering
    * this would be extremely expensive

In [45]:
@app.callback(Output("tab_content", "children"), [Input("tabs", "active_tab")])
def render_tab_content(active_tab):
    if active_tab is not None:
        if active_tab == "hist":
            return histograms
        elif active_tab == "polar":
            return polar_plots
        elif active_tab == "2D_hist":
            return hists_2D
        elif active_tab == "images":
            return images
    return html.H1(["No Tab Selected"])

## Run App

In [46]:
if __name__ == '__main__':
    app.run_server(debug=False)  

Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



INFO:__main__: Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.



INFO:__main__: in production, use a production WSGI server like gunicorn instead.



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [03/Aug/2020 14:20:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2020 14:20:50] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_5_1m1593119000.8.7.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2020 14:20:50] "GET /_dash-component-suites/dash_renderer/react@16.v1_5_1m1593119000.13.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2020 14:20:50] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_5_1m1593119000.13.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2020 14:20:50] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_5_1m1593119000.7.2.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2020 14:20:50] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v0_10_3m1593163089.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2020 14:20:50] "GET /_das